In [1]:
import pandas as pd
from os import listdir
from os.path import join
from tqdm import tqdm
from my_funcs import tokenize_es_a, tokenize_es_b, tokenize_es_c, tokenize_es_d 

In [2]:
folder = './datasets/'
for file in reversed(sorted(listdir(folder))): #nos uqedamos con el mas grande en caso de actualizar el ds
    if 'balanced' in file:
        print(f"ds_name = '{file}'")

ds_name = 'ds_semibalanced_1658_rs5150.csv'
ds_name = 'ds_semibalanced_1500_rs5150.csv'
ds_name = 'ds_balanced_1043_rs5150.csv'


In [3]:
ds_name = 'ds_balanced_1043_rs5150.csv'
#ds_name = 'ds_semibalanced_1500_rs5150.csv'
#ds_name = 'ds_semibalanced_1678_rs5150.csv'
print('selected: ', ds_name)

selected:  ds_balanced_1043_rs5150.csv


In [4]:
df = pd.read_csv(join(folder, ds_name))
pd.set_option('display.max_colwidth', 200)
print(df.columns)
df[['title', 'content']].head()

Index(['id', 'title', 'content', 'category'], dtype='object')


,title,content
0,Disneyland abre sus puertas como centro vacunatorio masivo contra el coronavirus,"Disneyland abre sus puertas como centro vacunatorio masivo contra el coronavirus. El parque de atracciones Disneyland en Anaheim, California, ha estado cerrado desde mediado de marzo por el alza d..."
1,México autoriza vacuna de Pfizer y es el primer país de Latinoamérica en aprobarla: Inoculación comenzaría este mes,México autoriza vacuna de Pfizer y es el primer país de Latinoamérica en aprobarla: Inoculación comenzaría este mes. La Comisión Federal para la Protección contra Riesgos Sanitarios (Cofepris) del...
2,El mega operativo de seguridad en Washington por la investidura de Biden: Será la ceremonia con más militares de la historia,"El mega operativo de seguridad en Washington por la investidura de Biden: Será la ceremonia con más militares de la historia. En Washington, un amplio perímetro que ocupa todo el centro, incluid..."
3,"""Swing states"" o estados bisagras: los terrenos claves donde se definirá la presidencial de EEUU","""Swing states"" o estados bisagras: los terrenos claves donde se definirá la presidencial de EEUU. Estados Unidos está ad portas de elegir a quien dirigirá los destinos del país por los próximos cu..."
4,"Merkel defiende nuevas restricciones y advierte sobre ""mentiras"" en lucha contra el covid","Merkel defiende nuevas restricciones y advierte sobre ""mentiras"" en lucha contra el covid. La Canciller alemana Angela Merkel defendió este jueves las nuevas medidas restrictivas anunciadas por ..."


In [5]:
MAX_LEN = 1600
DROPOUT = 0.5
MAX_VOCAB_SIZE = 50000
RANDOM_STATE = 8008
BATCH_SIZE = 64
EMBEDDING_DIM = 300
N_FILTERS = 100
FILTER_SIZES = [7, 7, 7]
LR = 0.0007
N_EPOCHS = 10
FREEZE_EMBEDDINGS = True
tokenizer = tokenize_es_c
TOKENIZER = tokenizer.__name__

## Truncar contenido
### Cuanto?

In [6]:
df['content'] = df.content.str.slice(0, MAX_LEN)
print('min len: ', df.content.map(len).min())
print('max len: ', df.content.map(len).max())

min len:  147
max len:  1600


In [7]:
df = df[['id', 'title', 'content', 'category']]
df

,id,title,content,category
0,6633017,Disneyland abre sus puertas como centro vacunatorio masivo contra el coronavirus,"Disneyland abre sus puertas como centro vacunatorio masivo contra el coronavirus. El parque de atracciones Disneyland en Anaheim, California, ha estado cerrado desde mediado de marzo por el alza d...",1-Mundo
1,14793496,México autoriza vacuna de Pfizer y es el primer país de Latinoamérica en aprobarla: Inoculación comenzaría este mes,México autoriza vacuna de Pfizer y es el primer país de Latinoamérica en aprobarla: Inoculación comenzaría este mes. La Comisión Federal para la Protección contra Riesgos Sanitarios (Cofepris) del...,1-Mundo
2,14786791,El mega operativo de seguridad en Washington por la investidura de Biden: Será la ceremonia con más militares de la historia,"El mega operativo de seguridad en Washington por la investidura de Biden: Será la ceremonia con más militares de la historia. En Washington, un amplio perímetro que ocupa todo el centro, incluid...",1-Mundo
3,621359,"""Swing states"" o estados bisagras: los terrenos claves donde se definirá la presidencial de EEUU","""Swing states"" o estados bisagras: los terrenos claves donde se definirá la presidencial de EEUU. Estados Unidos está ad portas de elegir a quien dirigirá los destinos del país por los próximos cu...",1-Mundo
4,14797573,"Merkel defiende nuevas restricciones y advierte sobre ""mentiras"" en lucha contra el covid","Merkel defiende nuevas restricciones y advierte sobre ""mentiras"" en lucha contra el covid. La Canciller alemana Angela Merkel defendió este jueves las nuevas medidas restrictivas anunciadas por ...",1-Mundo
...,...,...,...,...
10425,175524,Intendenta de Coquimbo rechaza posible fraude en Papaya Gate: No existe ninguna irregularidad,Intendenta de Coquimbo rechaza posible fraude en Papaya Gate: No existe ninguna irregularidad. Durante este jueves la Unidad de Investigación de Radio Bío Bío dio a conocer un proceso de compra de...,9-Crimen_delitos_y_Justicia
10426,24309,Vecinos de Maipú anuncian manifestaciones tras doble homicidio en feria navideña,"Vecinos de Maipú anuncian manifestaciones tras doble homicidio en feria navideña. A balazos y desde un auto en movimiento, fueron asesinados dos hombres en medio de un feria navideña repleta de ge...",9-Crimen_delitos_y_Justicia
10427,60339,"Matthei y polémicos dichos por defensores públicos: ""Se han convertido en enemigos de la ciudadanía""","Matthei y polémicos dichos por defensores públicos: ""Se han convertido en enemigos de la ciudadanía"". La alcaldesa de Providencia, Evelyn Matthei, se refirió en duros términos a la labor de los de...",9-Crimen_delitos_y_Justicia
10428,13408771,Avanza al Senado el proyecto de ley que otorga reserva de identidad a denunciantes de corrupción,"Avanza al Senado el proyecto de ley que otorga reserva de identidad a denunciantes de corrupción. Este martes, la Cámara de Diputadas y Diputados despachó al Senado el proyecto de ley del Minist...",9-Crimen_delitos_y_Justicia


In [8]:
from sklearn.utils import shuffle
df = shuffle(df, random_state=RANDOM_STATE)

In [9]:
import numpy as np
from sklearn.model_selection import StratifiedShuffleSplit

split = StratifiedShuffleSplit(n_splits=1, test_size=0.30, random_state=RANDOM_STATE)
for train_index, test_valid_index in split.split(df, df.category):
    train = df.iloc[train_index]
    test_valid_set = df.iloc[test_valid_index]

split2 = StratifiedShuffleSplit(n_splits=1, test_size=0.50, random_state=RANDOM_STATE)
for test_index, valid_index in split2.split(test_valid_set, test_valid_set.category):
    test = test_valid_set.iloc[test_index]
    valid = test_valid_set.iloc[valid_index]


#valid, test, train = np.split(df, [ int(.15*len(df)), int(.3*len(df))] )

In [10]:
print(df.shape)
print(train.shape)
print(valid.shape)
print(test.shape)

(10430, 4)
(7301, 4)
(1565, 4)
(1564, 4)


In [11]:
display(train.groupby(['category']).size())
display(valid.groupby(['category']).size())
display(test.groupby(['category']).size())

category
1-Mundo                        730
10-Salud                       730
2-Economía                     730
3-Política_y_Conflictos        730
4-Ciencias_y_Tecnología        730
5-Catástrofes_y_Accidentes     730
6-Cultura_y_Artes              730
7-Deporte                      731
8-Ecología_y_Planeta           730
9-Crimen_delitos_y_Justicia    730
dtype: int64

category
1-Mundo                        156
10-Salud                       156
2-Economía                     157
3-Política_y_Conflictos        157
4-Ciencias_y_Tecnología        157
5-Catástrofes_y_Accidentes     156
6-Cultura_y_Artes              157
7-Deporte                      156
8-Ecología_y_Planeta           157
9-Crimen_delitos_y_Justicia    156
dtype: int64

category
1-Mundo                        157
10-Salud                       157
2-Economía                     156
3-Política_y_Conflictos        156
4-Ciencias_y_Tecnología        156
5-Catástrofes_y_Accidentes     157
6-Cultura_y_Artes              156
7-Deporte                      156
8-Ecología_y_Planeta           156
9-Crimen_delitos_y_Justicia    157
dtype: int64

In [12]:
test.category.unique()

array(['8-Ecología_y_Planeta', '7-Deporte', '6-Cultura_y_Artes',
       '2-Economía', '3-Política_y_Conflictos', '1-Mundo',
       '9-Crimen_delitos_y_Justicia', '10-Salud',
       '5-Catástrofes_y_Accidentes', '4-Ciencias_y_Tecnología'],
      dtype=object)

In [13]:
train.to_csv("CNN_train.csv", encoding="UTF-8",index=False)
valid.to_csv("CNN_valid.csv", encoding="UTF-8",index=False)
test.to_csv("CNN_test.csv", encoding="UTF-8",index=False)

In [14]:
import torch
import random
import torchtext
from torchtext import data
from torchtext.legacy import data

In [15]:
#!python3 -m spacy download es_core_news_sm

import spacy
spacy_es = spacy.load('es_core_news_md')
#spacy_es = spacy.load('es_core_news_sm')

In [16]:

TEXT = data.Field(tokenize=tokenizer, batch_first = True)
CATEGORY = data.LabelField()  # MULTICLASS -se borró el argumento "(dtype = torch.float)"

In [17]:
fields = [(None, None),(None, None),('content', TEXT),('category', CATEGORY)]

In [ ]:
%%time
import numpy as np

SEED = RANDOM_STATE

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

train_data, valid_data, test_data = data.TabularDataset.splits(
                                        path = '.',
                                        train = 'CNN_train.csv',
                                        validation= 'CNN_valid.csv',
                                        test = 'CNN_test.csv',
                                        format = 'csv',
                                        fields = fields,
                                        skip_header = True
)

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# device = torch.device('cpu')
DEVICE_NAME = str(device)

print(DEVICE_NAME)

train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
                                                    (train_data, valid_data, test_data), 
                                                    batch_size = BATCH_SIZE, 
                                                    device = device,
                                                    sort_key=lambda x:len(x.category),
                                                    sort_within_batch=False)

In [ ]:
#!wget http://dcc.uchile.cl/~jperez/word-embeddings/glove-sbwc.i25.vec.gz

In [ ]:
## TENER VECTORES EN ESPAÑOL
vec = torchtext.vocab.Vectors('glove-sbwc.i25.vec.gz', cache='.')
TEXT.build_vocab(train_data,
                 vectors=vec,
                 max_size=MAX_VOCAB_SIZE,
                 unk_init=torch.Tensor.normal_)

In [ ]:
CATEGORY.build_vocab(train_data)

In [ ]:
cat_map = { v: k for k, v in CATEGORY.vocab.stoi.items()}
cat_map

In [ ]:
import torch.nn as nn
import torch.nn.functional as F

class CNN(nn.Module):
    def __init__(self, vocab_size, embedding_dim, n_filters, filter_sizes, output_dim, 
                 dropout, pad_idx):
        
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.convs = nn.ModuleList([
                                    nn.Conv1d(in_channels = 1, 
                                              out_channels = n_filters, 
                                              kernel_size = (fs, embedding_dim)) 
                                    for fs in filter_sizes
                                    ])
        
        self.fc = nn.Linear(len(filter_sizes) * n_filters, output_dim)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, text):
        
        #text = [sent len, batch size]
        
        #text = text.permute(1, 0)
                
        #text = [batch size, sent len]
        
        embedded = self.embedding(text)
                
        #embedded = [batch size, sent len, emb dim]
        
        embedded = embedded.unsqueeze(1)
        
        #embedded = [batch size, 1, sent len, emb dim]
        
        conved = [F.relu(conv(embedded)).squeeze(3) for conv in self.convs]
            
        #conv_n = [batch size, n_filters, sent len - filter_sizes[n]]
        
        pooled = [F.max_pool1d(conv, conv.shape[2]).squeeze(2) for conv in conved]
        
        #pooled_n = [batch size, n_filters]
        
        cat = self.dropout(torch.cat(pooled, dim = 1))

        #cat = [batch size, n_filters * len(filter_sizes)]
        
        return self.fc(cat)

In [ ]:
PAD_IDX = TEXT.vocab.stoi[TEXT.pad_token]
INPUT_DIM = len(TEXT.vocab)
OUTPUT_DIM = len(CATEGORY.vocab) ##### MULTICLASS ---> la dimensión del output no es 1 (clasificación binaria)



model = CNN(INPUT_DIM, EMBEDDING_DIM, N_FILTERS, FILTER_SIZES, OUTPUT_DIM, DROPOUT, PAD_IDX)
model = model.to(device)

In [ ]:
OUTPUT_DIM

In [ ]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

In [ ]:
pretrained_embeddings = TEXT.vocab.vectors
UNK_IDX = TEXT.vocab.stoi[TEXT.unk_token]

model.embedding.weight.data.copy_(pretrained_embeddings)
model.embedding.weight.data[UNK_IDX] = torch.zeros(EMBEDDING_DIM)
model.embedding.weight.data[PAD_IDX] = torch.zeros(EMBEDDING_DIM)
model.embedding.weight.requires_grad = FREEZE_EMBEDDINGS

In [ ]:
import torch.optim as optim

optimizer = optim.Adam(model.parameters(), lr=LR)

criterion = nn.CrossEntropyLoss() #MULTICLASS ---> en lugar de .BCEWithLogitsLoss() (Binary Cross Entropy)


criterion = criterion.to(device)

In [ ]:
OPTIMIZER = type(optimizer).__name__
CRITERION = type(criterion).__name__

In [ ]:
def train(model, iterator, optimizer, criterion, divisor):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.train()
    
    for batch in tqdm(iterator, desc='train'):
        
        optimizer.zero_grad()
        
        #print("Tamaño texto de entrada:"+str(batch.body.shape))
        
        predictions = model(batch.content)
        
        #print("Tamaño predecciones de salida:"+str(predictions.shape)) 
        
        #print("Tamaño target:"+str(batch.category.shape)) 
        
        loss = criterion(predictions, batch.category)
        
        acc = categorical_accuracy(predictions, batch.category, divisor)
        
        loss.backward()
        
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [ ]:
import time

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [ ]:
def categorical_accuracy(preds, y, divisor):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """
    max_preds = preds.argmax(dim=1, keepdim=True) # get the index of the max probability
    correct = max_preds.squeeze(1).eq(y)
    return correct.sum() / divisor([y.shape[0]])

In [ ]:
def evaluate(model, iterator, criterion, divisor):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval()
    
    with torch.no_grad():
    
        for batch in tqdm(iterator, desc='eval'):

            predictions = model(batch.content)
            loss = criterion(predictions, batch.category)
            
            acc = categorical_accuracy(predictions, batch.category, divisor)

            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)



In [ ]:
divisor = torch.FloatTensor if str(device) == 'cpu' else torch.cuda.FloatTensor

In [ ]:
from datetime import datetime
from os import remove

print("inicio optimización")

best_valid_loss = float('inf')
last_best = None
OVERWRITE_MODEL_FILE = True
#for epoch in range(N_EPOCHS):
for epoch in range(N_EPOCHS):

    start_time = time.time()
    train_loss, train_acc = train(model, train_iterator, optimizer, criterion, divisor)
    valid_loss, valid_acc = evaluate(model, valid_iterator, criterion, divisor)
    
    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    

    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        best_epoch = epoch + 1
        dt = str(datetime.now()).replace(' ', '_')
        name = f'./models/CNN_ep{str(best_epoch)}_{dt}.pt'
        torch.save({'epoca': epoch,
                    'model_state_dict': model.state_dict(),
                    'optimizer_state_dict': optimizer.state_dict(),
                    'Valid_loss': best_valid_loss}, name)
        if OVERWRITE_MODEL_FILE and last_best is not None:
            remove(last_best)
            last_best = name
        elif OVERWRITE_MODEL_FILE and last_best is None:
            last_best = name
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

In [ ]:
name, best_epoch

In [ ]:
best_model = CNN(INPUT_DIM,
                 EMBEDDING_DIM,
                 N_FILTERS,
                 FILTER_SIZES,
                 OUTPUT_DIM,
                 DROPOUT,
                 PAD_IDX)

In [ ]:
pretrained_embeddings = TEXT.vocab.vectors
UNK_IDX = TEXT.vocab.stoi[TEXT.unk_token]

best_model.embedding.weight.data.copy_(pretrained_embeddings)
best_model.embedding.weight.data[UNK_IDX] = torch.zeros(EMBEDDING_DIM)
best_model.embedding.weight.data[PAD_IDX] = torch.zeros(EMBEDDING_DIM)

In [ ]:
# name = './models/CNN_ep8_2021-12-13_12:56:03.466749.pt'

In [ ]:
best_model.load_state_dict(torch.load(name,
                                      map_location=device)['model_state_dict'])

In [ ]:
best_model = best_model.to(device)

In [ ]:
best_model.eval()
test_loss, test_acc = evaluate(best_model, test_iterator, criterion, divisor)

print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}%')

In [ ]:
best_model = best_model.cpu()
prediction_test = []
labels_test=[]
for batch in tqdm(test_iterator, desc='predictions'):
    labels_test.append(batch.category.cpu().detach().numpy())
    predictions = best_model(batch.content.cpu()).squeeze(1)
    prediction_test.append(predictions.argmax(dim=1).detach().numpy())
    
y_true = np.concatenate(labels_test)
y_pred = np.concatenate(prediction_test)

In [ ]:
lbs = list(cat_map.values())
lbs

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report
%matplotlib notebook
print('Resultados clasificación')
print('CNN')
print(classification_report(y_true,
                            y_pred,
                            target_names=lbs,))

In [ ]:
from my_funcs import plot_confusion_matrix
%matplotlib notebook

In [ ]:
cm = confusion_matrix(y_true=y_true, y_pred=y_pred)
plot_confusion_matrix(cm, labels=lbs)

In [ ]:
def predict_class(model, sentence, min_len = 4):

    tokenized = tokenizer(sentence)
    
    if len(tokenized) < min_len:
        tokenized += ['<pad>'] * (min_len - len(tokenized))
    device = 'cpu'
    indexed = [TEXT.vocab.stoi[t] for t in tokenized]
    
    tensor = torch.LongTensor(indexed).to(device)
    tensor = tensor.unsqueeze(0)
    preds = model(tensor)
    print(preds)
    max_preds = preds.argmax(dim = 1)
    return max_preds.item()

In [ ]:
noticia = """Sin embargo, estos experimentos se llevaban a cabo en secreto, y cuando otras secciones de Guardian se dieron cuenta, el proyecto se canceló de inmediato."""
cat_map.get(predict_class(best_model, noticia))

In [ ]:
cat_map.get(predict_class(best_model, "alexis y la copa america le pegoOoOooO"))

In [ ]:
noticia = """Artistas mexicanos retoman los espacios públicos a través de la danza. Más de 120 artistas de 16 estados de México se reúnen para dar vida al Encuentro Nacional de Danza 2021. El evento cultural se lleva a cabo en distintos puntos de la capital del país, entre ellos el Complejo Cultural Los Pinos.
"""
cat_map.get(predict_class(best_model, noticia))

# Comparar hiperparametros

gaurdar parametros utilizados y algunas metricas en tabla resumen

In [ ]:
wanted_keys = ['test_loss', 'test_acc', 'best_valid_loss', 'LR', 'ds_name', 'MAX_LEN', 'DROPOUT', 
               'N_EPOCHS', 'best_epoch', 'dt', 'TOKENIZER', 'DEVICE_NAME', 'MAX_VOCAB_SIZE',
               'RANDOM_STATE', 'BATCH_SIZE', 'EMBEDDING_DIM', 'N_FILTERS', 'FILTER_SIZES',
               'OPTIMIZER', 'CRITERION', 'FREEZE_EMBEDDINGS',
              ]

In [ ]:
from operator import itemgetter
wanted_values = list(itemgetter(*wanted_keys)(globals()))
hparams = dict(zip(wanted_keys, wanted_values))
saved = False
hparams

In [ ]:
import pickle
try:
    with open ('./cnn_info', 'rb') as fp:
        attemps = pickle.load(fp)
    if not saved:
        attemps.append(hparams)
        saved = True
except FileNotFoundError:
    print('cnn_info not found...')
    attemps = [hparams]
finally:
    with open('./cnn_info', 'wb') as fp:
        pickle.dump(attemps, fp)

In [ ]:
pd.DataFrame(attemps[-10:], columns=wanted_keys[:4]+['TOKENIZER', 'MAX_LEN', 'ds_name'])

In [ ]:
freno()

In [ ]:
wanted_keys = ['test_loss', 'test_acc', 'best_valid_loss', 'LR', 'ds_name', 'MAX_LEN', 'DROPOUT', 
               'N_EPOCHS', 'best_epoch', 'dt', 'DEVICE_NAME', 'MAX_VOCAB_SIZE',
               'RANDOM_STATE', 'BATCH_SIZE', 'EMBEDDING_DIM', 'N_FILTERS', 'FILTER_SIZES',
               'OPTIMIZER', 'CRITERION', 'FREEZE_EMBEDDINGS',
              ]

In [ ]:
import pickle
import pandas as pd
with open ('./cnn_info', 'rb') as fp:
    attemps = pickle.load(fp)
pd.DataFrame(attemps[-9:], columns=wanted_keys)